In [1]:
from os import walk
import pandas as pd
import numpy as np

In [ ]:
class employeeFile():
    def __init__(self):
        self.fileType
        
    
    

In [2]:
rawContentDf = pd.DataFrame(columns=['firstName', 'lastName','files'])

In [3]:
content = walk("active")
for root, dirs, files in content:
    dirHirarchy = root.split('\\')
    if len(dirHirarchy) == 3 : 
        fullName = dirHirarchy[1].split(', ')
        firstName = fullName[1]
        lastName = fullName[0]
        
        currentContent = pd.DataFrame({'firstName': firstName, 'lastName': lastName, 'files': [files]}, columns=['firstName', 'lastName', 'files'])
        rawContentDf = pd.concat([rawContentDf,currentContent], ignore_index=True)

In [4]:
rawContentDf

,firstName,lastName,files
0,firstname1,Lastname1,"[medTest2020.pdf, otherMedTest2021.pdf]"
1,firstname1,Lastname1,"[documentation2020.pdf, info2022.pdf, test2023..."
2,firstname2,Lastname2,"[medTest2020.pdf, otherMedTest2021.pdf]"
3,firstname2,Lastname2,"[documentation2020.pdf, info2022.pdf, test2023..."


In [5]:
rawContentDf.groupby(['firstName', 'lastName']).agg({'files': lambda x: list(np.concatenate(x.to_numpy()))}).reset_index()

,firstName,lastName,files
0,firstname1,Lastname1,"[medTest2020.pdf, otherMedTest2021.pdf, docume..."
1,firstname2,Lastname2,"[medTest2020.pdf, otherMedTest2021.pdf, docume..."
